# Prerequisites

The following steps must be satisfied before running the rest of the notebook:

- Run the `Augment_Data` notebook in order to generate the training files and images that are referred to throughout the notebook.
- Make changes to `hm_convert` to read the added files/folders when running `build_dataset`.

Throughout the notebook, directory paths are used for references that depend on your local setup. There are comments that describe what each path should be pointing to so that should help set up the notebook to work for your hierarchy.

# Setup

In [1]:
### Mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# local mmf installation
%cd '/content/drive/MyDrive/mmf-main'
!pip install -e .

/content/drive/MyDrive/mmf-main
Obtaining file:///content/drive/MyDrive/mmf-main
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for mmf (pyproject.toml) ... done
  Created wheel for mmf: filename=mmf-1.0.0rc12-0.editable-cp310-cp310-linux_x86_64.whl size=10743 sha256=e83c4fc145a61e52a89dab1f3d1bece658b4e007be246b6b0b9938e50a2fab93
  Stored in directory: /tmp/pip-ephem-wheel-cache-zs1ovasu/wheels/e3/9f/fc/d49548515b10b7b3c02969460b19f70eb734152532b702074e
Successfully built mmf
  Attempting uninstall: mmf
    Found existing installation: mmf 1.0.0rc12
    Uninstalling mmf-1.0.0rc12:
      Successfully uninstalled mmf-1.0.0rc12


In [3]:
### Import General Modules, 3min
from mmf.common.registry import registry
from mmf.models.mmbt import MMBT
from mmf.utils.build import build_dataset
from mmf.utils.env import setup_imports
import matplotlib.pyplot as plt
from PIL import Image
import torch

### Import Modules for Fusion Model
# All model using MMF need to inherit BaseModel
from mmf.models.base_model import BaseModel
# Builder methods for image encoder and classifier
from mmf.utils.build import build_classifier_layer, build_image_encoder,build_text_encoder

setup_imports()

# !mmf_convert_hm --zip_file="/content/drive/MyDrive/hateful-memes/hateful_memes.zip" --bypass_checksum=1
# dataset = build_dataset("hateful_memes")

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [9]:
!ls ../hateful_memes/datasets

coco  gqa  hateful_memes.zip  mmimdb  vizwiz


In [8]:
### Build Dataset
# provide the path to the compressed dataset folder (should have hateful_memes as root folder when uncompressed)
!mmf_convert_hm --zip_file="../hateful_memes/datasets/hateful_memes.zip" --bypass_checksum=1
dataset = build_dataset("hateful_memes")

2024-04-25 21:51:38.115248: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 21:51:38.115299: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 21:51:38.116894: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-25 21:51:39.165749: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-25 21:51:43.397195: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.10

/root/.cache/torch/mmf/glove.6B.zip: 862MB [02:41, 5.33MB/s]                           
100%|█████████▉| 399999/400000 [00:47<00:00, 8455.96it/s]


# Training

The sections below rely on training `json` files that were created in a separate notebook in the Prerequisites section. If you see a warning about these files not existing please make sure to follow the steps in that section.

## Pre-trained with CC

In [34]:
!ls projects/visual_bert/configs/hateful_memes/

gqa		      masked_coco		  masked_sbu   nlvr2		  vqa2
hateful_memes	      masked_conceptual_captions  masked_vqa2  visual_entailment
localized_narratives  masked_gqa		  mmimdb       vizwiz


### Dataset: Default

In [35]:
!mmf_run config="projects/visual_bert/configs/hateful_memes/defaults.yaml" \
  model=visual_bert \
  dataset=hateful_memes \
  run_type=train_val \
  training.log_interval=200 \
  training.max_updates=1500 \
  training.batch_size=64 \
  training.evaluation_interval=200 \
  training.tensorboard=True \
  training.checkpoint_interval=200 \
  checkpoint.resume_pretrained=True \
  checkpoint.resume_zoo=visual_bert.pretrained.cc \
  dataset_config.hateful_memes.annotations.train[0]="hateful_memes/defaults/annotations/train.jsonl" \
  dataset_config.hateful_memes.annotations.val[0]="hateful_memes/defaults/annotations/dev_unseen.jsonl" \
  dataset_config.hateful_memes.annotations.test[0]="hateful_memes/defaults/annotations/test_unseen.jsonl"

2024-04-25 22:21:02.827704: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 22:21:02.827764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 22:21:02.829491: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-25 22:21:03.872399: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-25 22:21:07.923996: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.10

### Dataset: Appended with images from dev

In [38]:
!mmf_run config="projects/visual_bert/configs/hateful_memes/defaults.yaml" \
  model=visual_bert \
  dataset=hateful_memes \
  run_type=train_val \
  training.log_interval=200 \
  training.batch_size=64 \
  training.evaluation_interval=200 \
  training.tensorboard=True \
  training.checkpoint_interval=200 \
  checkpoint.resume_pretrained=True \
  checkpoint.resume_zoo=visual_bert.pretrained.cc \
  dataset_config.hateful_memes.annotations.train[0]="hateful_memes/defaults/annotations/train_dev.jsonl" \
  dataset_config.hateful_memes.annotations.val[0]="hateful_memes/defaults/annotations/dev_unseen.jsonl" \
  dataset_config.hateful_memes.annotations.test[0]="hateful_memes/defaults/annotations/test_unseen.jsonl"

2024-04-25 23:55:22.242318: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 23:55:22.242373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 23:55:22.244068: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-25 23:55:23.324846: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-25 23:55:27.801612: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.10

### Dataset: Augmented with partial transformations

In [39]:
!mmf_run config="projects/visual_bert/configs/hateful_memes/defaults.yaml" \
  model=visual_bert \
  dataset=hateful_memes \
  run_type=train_val \
  training.log_interval=200 \
  training.batch_size=64 \
  training.evaluation_interval=200 \
  training.tensorboard=True \
  training.checkpoint_interval=200 \
  checkpoint.resume_pretrained=True \
  checkpoint.resume_zoo=visual_bert.pretrained.cc \
  dataset_config.hateful_memes.annotations.train[0]="hateful_memes/defaults/annotations/train_dev_transform_half.jsonl" \
  dataset_config.hateful_memes.annotations.val[0]="hateful_memes/defaults/annotations/dev_unseen.jsonl" \
  dataset_config.hateful_memes.annotations.test[0]="hateful_memes/defaults/annotations/test_unseen.jsonl"

2024-04-26 00:16:25.366655: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 00:16:25.366712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 00:16:25.368375: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-26 00:16:26.519990: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-26 00:16:31.620813: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.10

### Dataset: Augmented with complete transformations

In [40]:
!mmf_run config="projects/visual_bert/configs/hateful_memes/defaults.yaml" \
  model=visual_bert \
  dataset=hateful_memes \
  run_type=train_val \
  training.log_interval=200 \
  training.batch_size=64 \
  training.evaluation_interval=200 \
  training.tensorboard=True \
  training.checkpoint_interval=200 \
  checkpoint.resume_pretrained=True \
  checkpoint.resume_zoo=visual_bert.pretrained.cc \
  dataset_config.hateful_memes.annotations.train[0]="hateful_memes/defaults/annotations/train_dev_transform_all.jsonl" \
  dataset_config.hateful_memes.annotations.val[0]="hateful_memes/defaults/annotations/dev_unseen.jsonl" \
  dataset_config.hateful_memes.annotations.test[0]="hateful_memes/defaults/annotations/test_unseen.jsonl"

2024-04-26 00:39:59.595549: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 00:39:59.595599: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 00:39:59.597252: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-26 00:40:00.656791: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-26 00:40:04.875685: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.10

### Dataset: Augmented with combined transformations

In [41]:
!mmf_run config="projects/visual_bert/configs/hateful_memes/defaults.yaml" \
  model=visual_bert \
  dataset=hateful_memes \
  run_type=train_val \
  training.log_interval=200 \
  training.batch_size=64 \
  training.evaluation_interval=200 \
  training.tensorboard=True \
  training.checkpoint_interval=200 \
  checkpoint.resume_pretrained=True \
  checkpoint.resume_zoo=visual_bert.pretrained.cc \
  dataset_config.hateful_memes.annotations.train[0]="hateful_memes/defaults/annotations/train_dev_transform_all_combined.jsonl" \
  dataset_config.hateful_memes.annotations.val[0]="hateful_memes/defaults/annotations/dev_unseen.jsonl" \
  dataset_config.hateful_memes.annotations.test[0]="hateful_memes/defaults/annotations/test_unseen.jsonl"

2024-04-26 01:04:36.501174: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 01:04:36.501227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 01:04:36.502894: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-26 01:04:37.642815: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-26 01:04:42.690921: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.10